In [1]:
import mobster as mb

import numpy as np



Bad key "text.kerning_factor" on line 4 in
/Users/salvatore/miniconda3/envs/hmobster/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
data = mb.load_example_data("./")



In [3]:
fitted = mb.fit_mobster(data,2,0,purity = 0.95,  truncated_pareto=True, max_it = 200, lr = 0.01,  e = 0.000001, lrd_gamma = 0.1,   
        number_of_trials_k= 300, prior_lims_k=[0.1,10000], prior_lims_clonal=[0.1,10000.], number_of_trials_clonal_mean= 300.)

Running MOBSTER on 2 karyotypes with 2 subclones.
Fitting a model without tail

Computing cluster assignements.
Computing information criteria.
Done!


ELBO: -5284.579052702  : 100%|██████████| 200/200 [00:04<00:00, 43.36it/s]


In [4]:


mb.plot_results(data, fitted, output= "cane.png", drivers=None)

In [5]:
fitted["model_parameters"]["2:1"]



{'cluster_probs': array([[1.84872732e-07, 2.42316976e-01, 2.40424416e-11, ...,
         3.97846699e-01, 7.39545491e-12, 3.69519992e-09],
        [9.99999762e-01, 7.57654488e-01, 1.00000000e+00, ...,
         1.35921471e-16, 1.00000000e+00, 1.20259434e-41],
        [5.00170740e-30, 5.58710201e-19, 5.51846402e-37, ...,
         3.78905224e-06, 6.68485733e-38, 9.99997973e-01],
        [1.41790076e-13, 2.84890702e-05, 5.42620607e-19, ...,
         6.02149487e-01, 1.02620285e-19, 2.02035949e-06]], dtype=float32),
 'cluster_assignments': array(['C2', 'C2', 'C2', ..., 'S2', 'C2', 'S1'], dtype=object),
 'cluster_types': array(['C1', 'C2', 'S1', 'S2'], dtype='<U2'),
 'mixture_probs': array([0.3843188 , 0.39734086, 0.08379222, 0.13454814], dtype=float32),
 'beta_concentration1': array([44.780575, 84.10113 , 13.701508, 42.99995 ], dtype=float32),
 'beta_concentration2': array([ 95.30993,  55.98937, 126.38899, 120.55843], dtype=float32),
 'ccf_subclones': array([0.29370776, 0.7894978 ], dtype=floa